# Deep Learning with PyTorch: A 60 min blitz

## What is PyTorch?
It’s a Python based scientific computing package targeted at two sets of audiences:

- A replacement for numpy to use the power of GPUs
- a deep learning research platform that provides maximum flexibility and speed

## Getting Started
### Tensors
Tensors are similar to numpy’s ndarrays, with the addition being that Tensors can also be used on a GPU to accelerate computing.

In [1]:
from __future__ import print_function
import torch
from torch.autograd import Variable

In [2]:
x = torch.Tensor(5, 3); x


-1.7798e-33  4.5857e-41 -1.7798e-33
 4.5857e-41         nan  8.9135e-33
 1.1247e-08  1.0563e-05  1.7821e+02
 1.9419e+02  2.6515e-09  1.0609e-08
 7.8486e+02  7.0485e+02  1.0650e-32
[torch.FloatTensor of size 5x3]

In [3]:
x = torch.rand(5, 3); x


 0.5804  0.6874  0.5689
 0.5090  0.8797  0.9391
 0.7952  0.2505  0.8726
 0.8268  0.0662  0.1425
 0.8460  0.7079  0.8127
[torch.FloatTensor of size 5x3]

In [4]:
x.size()

torch.Size([5, 3])

In [5]:
y = torch.rand(5, 3)

In [6]:
x + y


 0.7379  0.8916  0.6659
 0.7262  1.6339  1.3564
 1.5982  0.6585  1.3398
 1.2081  0.3011  0.2486
 1.7776  0.8180  1.0094
[torch.FloatTensor of size 5x3]

In [7]:
torch.add(x, y)


 0.7379  0.8916  0.6659
 0.7262  1.6339  1.3564
 1.5982  0.6585  1.3398
 1.2081  0.3011  0.2486
 1.7776  0.8180  1.0094
[torch.FloatTensor of size 5x3]

In [8]:
result = torch.Tensor(5, 3)
torch.add(x, y, out=result); result


 0.7379  0.8916  0.6659
 0.7262  1.6339  1.3564
 1.5982  0.6585  1.3398
 1.2081  0.3011  0.2486
 1.7776  0.8180  1.0094
[torch.FloatTensor of size 5x3]

In [9]:
result2 = torch.add(x,y)

In [10]:
y.add_(x); y


 0.7379  0.8916  0.6659
 0.7262  1.6339  1.3564
 1.5982  0.6585  1.3398
 1.2081  0.3011  0.2486
 1.7776  0.8180  1.0094
[torch.FloatTensor of size 5x3]

Any operation that mutates a tensor in-place is post-fixed with an `_` For example: `x.copy_(y), x.t_()`, will change x.

In [11]:
# Numpy like indexing
x[:, 1]


 0.6874
 0.8797
 0.2505
 0.0662
 0.7079
[torch.FloatTensor of size 5]

## Numpy Bridge
Converting a torch Tensor to a numpy array and vice versa is a breeze.

The torch Tensor and numpy array will share their underlying memory locations, and changing one will change the other.

In [12]:
a = torch.ones(5); a


 1
 1
 1
 1
 1
[torch.FloatTensor of size 5]

In [13]:
b = a.numpy(); b

array([ 1.,  1.,  1.,  1.,  1.], dtype=float32)

In [14]:
a.add_(1); a


 2
 2
 2
 2
 2
[torch.FloatTensor of size 5]

In [15]:
b

array([ 2.,  2.,  2.,  2.,  2.], dtype=float32)

In [16]:
import numpy as np

In [17]:
a = np.ones(5); a

array([ 1.,  1.,  1.,  1.,  1.])

In [18]:
b = torch.from_numpy(a); b


 1
 1
 1
 1
 1
[torch.DoubleTensor of size 5]

In [19]:
np.add(a, 1, out=a)

array([ 2.,  2.,  2.,  2.,  2.])

In [20]:
b


 2
 2
 2
 2
 2
[torch.DoubleTensor of size 5]

All the Tensors on the CPU except a CharTensor support converting to NumPy and back.

## CUDA Tensors
Tensors can be moved onto GPU using the .cuda function.

In [21]:
if torch.cuda.is_available():
    x = x.cuda()
    y = y.cuda()
    print(x+y)


 1.3184  1.5789  1.2348
 1.2351  2.5136  2.2955
 2.3934  0.9089  2.2124
 2.0350  0.3672  0.3911
 2.6235  1.5259  1.8221
[torch.cuda.FloatTensor of size 5x3 (GPU 0)]



## Autograd: automatic differentiation

Central to all neural networks in PyTorch is the autograd package. Let’s first briefly visit this, and we will then go to training our first neural network.

The autograd package provides automatic differentiation for all operations on Tensors. It is a define-by-run framework, which means that your backprop is defined by how your code is run, and that every single iteration can be different.

Let us see this in more simple terms with some examples.

### Variable
`autograd.Variable` is the central class of the package. It wraps a Tensor, and supports nearly all of operations defined on it. Once you finish your computation you can call `.backward()` and have all the gradients computed automatically.

You can access the raw tensor through the `.data` attribute, while the gradient w.r.t. this variable is accumulated into `.grad`.

There’s one more class which is very important for autograd implementation - a `Function`.

`Variable` and `Function` are interconnected and build up an acyclic graph, that encodes a complete history of computation. Each variable has a `.grad_fn` attribute that references a `Function` that has created the `Variable` (except for Variables created by the user - their `grad_fn` is `None`).

If you want to compute the derivatives, you can call `.backward()` on a Variable. If `Variable` is a scalar (i.e. it holds a one element data), you don’t need to specify any arguments to backward(), however if it has more elements, you need to specify a `grad_output` argument that is a tensor of matching shape.

In [22]:
x = Variable(torch.ones(2,2), requires_grad=True); x

Variable containing:
 1  1
 1  1
[torch.FloatTensor of size 2x2]

In [23]:
y = x + 2; y

Variable containing:
 3  3
 3  3
[torch.FloatTensor of size 2x2]

In [25]:
x.data


 1  1
 1  1
[torch.FloatTensor of size 2x2]

In [26]:
print(x.grad_fn)

None


`y` was created as a result of an operation, so it has a `grad_fn`.

In [27]:
print(y.grad_fn)

In [28]:
z = y * y * 3
out = z.mean()
print(z, out)

Variable containing:
 27  27
 27  27
[torch.FloatTensor of size 2x2]
 Variable containing:
 27
[torch.FloatTensor of size 1]



In [29]:
out.backward()

In [30]:
print(x.grad)

Variable containing:
 4.5000  4.5000
 4.5000  4.5000
[torch.FloatTensor of size 2x2]



In [31]:
x = torch.randn(3)
x = Variable(x, requires_grad = True)
y = x * 2

In [32]:
while y.data.norm() < 1000:
    y = y * 2

print(y)

Variable containing:
 506.1036
 942.9279
-772.6771
[torch.FloatTensor of size 3]



In [33]:
gradients = torch.FloatTensor([0.1, 1.0, 0.0001])
y.backward(gradients)

print(x.grad)

Variable containing:
  102.4000
 1024.0000
    0.1024
[torch.FloatTensor of size 3]



## Neural Networks